# **Quantum Weirdness:  Exploring Quantum Error Correction**

## Circuit Setup, IBM Channel Setup, and Imports

### Load in IBM account

In [9]:
import os

# env variables so that my key doesn't get leaked
from dotenv import load_dotenv
load_dotenv()
token = os.getenv("API_KEY")

from qiskit_ibm_runtime import QiskitRuntimeService
 
# get my IBM acc
QiskitRuntimeService.save_account(
  token=token,
  channel="ibm_quantum", # `channel` distinguishes between different account types
  overwrite=True
)



### Imports

In [2]:
from pylab import *

from collections import Counter

from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler
from qiskit import QuantumCircuit, transpile
from qiskit import QuantumRegister, ClassicalRegister
from qiskit.quantum_info import Statevector
from qiskit.visualization import circuit_drawer, plot_histogram
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, pauli_error


# for MacOSX 
# https://stackoverflow.com/questions/63722669/matplotlib-animation-works-on-windows-and-linux-but-not-on-mac-os
matplotlib.use("TkAgg")

### [Test if IBM Channel created correctly](https://docs.quantum.ibm.com/guides/setup-channel)

In [ ]:
# Create empty circuit
example_circuit = QuantumCircuit(2)
example_circuit.measure_all()
 
# You'll need to specify the credentials when initializing QiskitRuntimeService, if they were not previously saved.
service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False)
 
sampler = Sampler(backend)
job = sampler.run([example_circuit])
print(f"job id: {job.job_id()}")
result = job.result()
print(result)

job id: cxb4fexztp30008g7460
PrimitiveResult([SamplerPubResult(data=DataBin(meas=BitArray(<shape=(), num_shots=4096, num_bits=2>)), metadata={'circuit_metadata': {}})], metadata={'execution': {'execution_spans': ExecutionSpans([SliceSpan(<start='2024-12-09 01:28:29', stop='2024-12-09 01:30:11', size=4096>)])}, 'version': 2})


### Making a quantum circuit and trying out different gates

The **Hadamard gate** has the matrix form of 
$$ H = \begin{pmatrix}
\frac{1}{\sqrt 2} & \frac{1}{\sqrt 2}
\\
\frac{1}{\sqrt 2} & -\frac{1}{\sqrt 2}
\end{pmatrix} 
$$

**Functionality**

Puts the qubit in a state of superposition


In [83]:
# Demo Hadamard Gate

# single qubit quantum circuit
qubit = QuantumRegister(1, name='q')
bit = ClassicalRegister(1, name='A') # 1 classical bit called A
hadamard_circuit = QuantumCircuit(qubit, bit)

# reassign to make a bit more clear
q0 = qubit
A = bit

state = Statevector.from_instruction(hadamard_circuit)
state.draw('bloch').show() # show the qubit in |0> state

hadamard_circuit.h(q0) # Hadamard on our single qubit

finalstate = Statevector.from_instruction(hadamard_circuit)
finalstate.draw('bloch').show() # show the qubit in superposition state

hadamard_circuit.measure(q0, bit)
hadamard_circuit.draw('mpl').show()



The **CNOT gate** operates on two qubits: one control and one target. Its matrix form is:

$$
CNOT = \begin{pmatrix}
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 0 & 1 \\
0 & 0 & 1 & 0
\end{pmatrix}
$$

**Functionality**
- If the **control qubit** is $ |0\rangle $, the target qubit remains unchanged.
- If the **control qubit** is $ |1\rangle $, the target qubit flips $ |0\rangle \leftrightarrow |1\rangle $ (Similar to applying the NOT operator to a bit).

**Example**
$$
\begin{array}{|c|c|}
\hline
\text{Input State} & \text{Output State} \\
\hline
|00\rangle & |00\rangle \\
|01\rangle & |01\rangle \\
|10\rangle & |11\rangle \\
|11\rangle & |10\rangle \\
\hline
\end{array}
$$

#### **Entanglement**
The CNOT gate can create entanglement when the control qubit is in a superposition state. For example:
- Starting with $ |+\rangle|0\rangle $, where $ |+\rangle = \frac{|0\rangle + |1\rangle}{\sqrt{2}} $, applying the CNOT gate produces the **Bell state**:
$$
\frac{|00\rangle + |11\rangle}{\sqrt{2}}
$$
which means that both qubits are entangled and have a $\frac 1 2 = 50\%$ chance to be in the 0 basis state or 1 basis state.


In [14]:
# Demo CNOT Gate

# create circuit

qubits = QuantumRegister(2)
bell_circuit = QuantumCircuit(qubits)
q0, q1 = qubits
bell_circuit.h(q0)  # Hadamard on qubit 0
bell_circuit.cx(q0, q1)  # CNOT with control=0, target=1

# display circuit and save as image (other way to draw circuit)
circuit_drawer(bell_circuit, output='mpl', filename='figures/qc.png').show()

# show vector representation of statevector using Statevector class in Qiskit
state = Statevector.from_instruction(bell_circuit)
state.draw('latex')


<IPython.core.display.Latex object>

### Implementing the error correction codes ([For Shor's Code](https://quantumcomputinguk.org/tutorials/quantum-error-correction-shor-code-in-qiskit))

In [ ]:
def encoding(qc, q0, q1, q2):
    """Encoding process, encode q0 state on other qubits

    Args:
        qc (QuantumCircuit): _description_
        q0 (_type_): Logical Qubit
        q1 (_type_): Ancilla Qubit 1
        q2 (_type_): Ancilla Qubit 2
    """
    qc.cx(0, 1)
    qc.cx(0, 2)

def decoding(qc, q0, q1, q2):
    """decoding process to reverse encoding

    Args:
        qc (QuantumCircuit): _description_
        q0 (_type_): Logical Qubit
        q1 (_type_): Ancilla Qubit 1
        q2 (_type_): Ancilla Qubit 2
    """
    qc.cx(0, 1)
    qc.cx(0, 2)
    # qc.ccx(2, 1, 0)

def simulate_error():
    """Bit-flip error toy model simulation from Qiskit documentation
    """
    # example error probabilities
    p_reset = 0.03
    p_meas = 0.1
    p_gate1 = 0.05

    # quantumError objects
    # error_reset = pauli_error([('X', p_reset), ('I', 1 - p_reset)]) #reset error 
    error_meas = pauli_error([('X',p_meas), ('I', 1 - p_meas)]) # measurement error
    error_gate1 = pauli_error([('X',p_gate1), ('I', 1 - p_gate1)]) # single-qubit gate error
    error_gate2 = error_gate1.tensor(error_gate1) #two-qubit gate error

    # add errors to noise model
    noise_bit_flip = NoiseModel()
    # noise_bit_flip.add_all_qubit_quantum_error(error_reset, "reset")
    noise_bit_flip.add_all_qubit_quantum_error(error_meas, "measure")
    noise_bit_flip.add_all_qubit_quantum_error(error_gate1, ["u1", "u2", "u3"])
    noise_bit_flip.add_all_qubit_quantum_error(error_gate2, ["cx"]) # errors on CNOT 

    print(noise_bit_flip)
    return noise_bit_flip

def three_bit_repetition_code():
    """Implementation of a 3-Bit Repetition code 

    Returns:
        QuantumCircuit: The circuit after measurement
    """
    # For 3-bit repetition code [[3, 1, 3]]
    circuit = QuantumCircuit(3) # 3 qubits 

    # encoding 2 ancillary qubits
    encoding(circuit, 0, 1, 2)

    # decoding, apply CNOTs on ancillary qubits again 
    decoding(circuit, 2, 1, 0)

    # toffoli gate (ONLY USE IF WE WANT TO PERFORM DIRECT MAJORITY VOTING AT THE CIRCUIT LEVEL)(Collapse result into single qubit)
    # circuit.ccx(2, 1, 0)

    # circuit.measure(0, 0) # measure logical qubit and save result in our classical bit

    # measure all qubits
    circuit.barrier()
    circuit.measure_all() # saves the results of the 3 qubits into 3 classical bits

    circuit.draw('mpl').show()

    return circuit

def no_error_correction():
    """Implementation of a 3-Qubit Circuit with no type of error correction

    Returns:
        QuantumCircuit: The circuit after measurement
    """
    circuit = QuantumCircuit(3) # 3 qubits

    # encoding 2 physical qubits
    encoding(circuit, 0, 1, 2)

    # measure all qubits
    circuit.measure_all()

    circuit.draw('mpl').show()

    return circuit

def shors_encoding(qc, logical_qubit, ancillas):
    """Encoding process for Shor's Code.

    Args:
        qc (QuantumCircuit): QuantumCircuit to encode on.
        logical_qubit (int): Index of the logical qubit.
        ancillas (list[int]): List of ancillary qubit indices.
    """
    # Bit-flip protection
    qc.cx(logical_qubit, ancillas[2])
    qc.cx(logical_qubit, ancillas[5])

    # Apply Hadamard to prepare for phase-flip protection
    qc.h(logical_qubit)
    qc.h(ancillas[2])
    qc.h(ancillas[5])

    # Encode phase-flip protection across each group
    # 1st block
    qc.cx(logical_qubit, ancillas[0])
    qc.cx(logical_qubit, ancillas[1])

    # 2nd block
    qc.cx(logical_qubit + 3, ancillas[3])
    qc.cx(logical_qubit + 3, ancillas[4])

    # 3rd block
    qc.cx(logical_qubit + 6, ancillas[6])
    qc.cx(logical_qubit + 6, ancillas[7])

def shors_decoding(qc, logical_qubit, ancillas):
    """Decoding process for Shor's Code.

    Args:
        qc (QuantumCircuit): QuantumCircuit to decode on.
        logical_qubit (int): Index of the logical qubit.
        ancillas (list[int]): List of ancillary qubit indices.
    """
    # Reverse phase-flip protection
    # 1st block
    qc.cx(logical_qubit, ancillas[0])
    qc.cx(logical_qubit, ancillas[1])

    # 2nd block
    qc.cx(logical_qubit + 3, ancillas[3])
    qc.cx(logical_qubit + 3, ancillas[4])

    # 3rd block
    qc.cx(logical_qubit + 6, ancillas[6])
    qc.cx(logical_qubit + 6, ancillas[7])
    
    # Toffoli gates
    qc.ccx(ancillas[0], ancillas[1], logical_qubit)
    qc.ccx(ancillas[3], ancillas[4], logical_qubit + 3)
    qc.ccx(ancillas[6], ancillas[7], logical_qubit + 6)

    # Undo Hadamard
    qc.h(logical_qubit)
    qc.h(ancillas[2])
    qc.h(ancillas[5])

    # Reverse bit-flip protection
    qc.cx(logical_qubit, ancillas[2])
    qc.cx(logical_qubit, ancillas[5])
    qc.ccx(ancillas[2], ancillas[5], logical_qubit)

def shors_code():
    """Implementation of Shor's Code with noise simulation.

    Returns:
        QuantumCircuit: Shor's Code circuit with measurement.
    """
    # 9 qubits: 1 logical + 8 ancillas, 1 classical bit for logical measurement
    qc = QuantumCircuit(9, 1)

    logical_qubit = 0
    ancillas = [1, 2, 3, 4, 5, 6, 7, 8]

    qc.initialize([1, 0], logical_qubit)

    # encoding: Encode the logical qubit into 9 qubits
    shors_encoding(qc, logical_qubit, ancillas)

    # for figure 
    # qc.barrier()
    # qc.x(4)  # Simulate a bit-flip error on qubit 4
    # qc.z(7)  # Simulate a phase-flip error on qubit 7
    # qc.barrier()

    # decoding: Reverse the encoding to recover the logical qubit
    shors_decoding(qc, logical_qubit, ancillas)

    # Measure the logical qubit
    qc.barrier
    qc.measure(0, 0)

    # for figure
    # qc.draw('mpl').show()

    return qc

def no_error_correction_shor():
    """Implementation of Shor's Code with noise simulation.

    Returns:
        QuantumCircuit: Shor's Code circuit with measurement.
    """
    # 9 qubits: 1 logical + 8 ancillas, 1 classical bit for logical measurement
    qc = QuantumCircuit(9, 1)

    logical_qubit = 0
    ancillas = [1, 2, 3, 4, 5, 6, 7, 8]

    qc.initialize([1, 0], logical_qubit)

    # Encoding: Encode the logical qubit into 9 qubits
    shors_encoding(qc, logical_qubit, ancillas)

    # for figure 
    # qc.barrier()
    # qc.x(4)  # Simulate a bit-flip error on qubit 4
    # qc.z(7)  # Simulate a phase-flip error on qubit 7
    # qc.barrier()

    # Measure the logical qubit
    qc.barrier
    qc.measure(0, 0)

    return qc

### Test the error correction codes against [simulated noise](https://qiskit.github.io/qiskit-aer/tutorials/3_building_noise_models.html)

In [13]:
qc_3bit = three_bit_repetition_code()
qc_nocode = no_error_correction()
noise_bit_flip = simulate_error()

# Transpile for simulator
simulator = AerSimulator(noise_model=noise_bit_flip)
circ = transpile(qc_3bit, simulator) 
circ1 = transpile(qc_nocode, simulator)

# Run and get counts for non-corrected circuit
result_uncorrected = simulator.run(circ1).result()
counts_uncorrected = result_uncorrected.get_counts(0)

# Run and get counts for corrected circuit
result_corrected = simulator.run(circ).result()
counts_corrected = result_corrected.get_counts(0)

# plot
fig, axes = subplots(1, 2, figsize=(10, 5))

plot_histogram(counts_uncorrected, ax=axes[0])
axes[0].set_title("No 3-Bit Repetition Code")

plot_histogram(counts_corrected, ax=axes[1])
axes[1].set_title("3-Bit Repetition Code")

tight_layout()
show()


NoiseModel:
  Basis gates: ['cx', 'id', 'rz', 'sx', 'u1', 'u2', 'u3']
  Instructions with noise: ['u1', 'u3', 'cx', 'u2', 'measure']
  All-qubits errors: ['measure', 'u1', 'u2', 'u3', 'cx']


### Test Shor's Code

In [10]:
# Ideal simulator and execution
circ_shor = shors_code()
circ_no_shor = no_error_correction_shor()

sim_ideal = AerSimulator()

result_ideal = sim_ideal.run(circ_shor).result()
counts_shor = result_ideal.get_counts(0)

result_ideal1 = sim_ideal.run(circ_no_shor).result()
counts_no_shor = result_ideal1.get_counts(0)

# plot
fig, axes = subplots(1, 2, figsize=(10, 5))

plot_histogram(counts_no_shor, ax=axes[0])
axes[0].set_title("No Shor's Code")

plot_histogram(counts_shor, ax=axes[1])
axes[1].set_title("With Shor's Code")

tight_layout()
show()

## Run the error correction codes on quantum hardware

#### Without Shor's Code

In [7]:
qc_shor = shors_code()
qc_no_shor = no_error_correction_shor()

service = QiskitRuntimeService()
backend = service.least_busy(min_num_qubits=9, operational=True, simulator=False)
print(backend)

# transpile the circuit for the backend
transpiled_circuit = transpile(qc_no_shor, backend, optimization_level=2)

job = backend.run(transpiled_circuit)
result = job.result()
real_counts_no_shor = result.get_counts()
print(real_counts_no_shor)

<IBMBackend('ibm_brisbane')>


/var/folders/24/ggylxxcj3xq362xbyt3gkg_00000gn/T/ipykernel_41701/18902787.py:11: DeprecationWarning: backend.run() and related sessions methods are deprecated  as of qiskit-ibm-runtime 0.23 and will be removed no sooner than 6 months after the release date. More details can be found in the primitives migration guide https://docs.quantum.ibm.com/migration-guides/qiskit-runtime.
  job = backend.run(transpiled_circuit)


{'0': 1948, '1': 2052}


#### With Shor's Code

In [8]:
service = QiskitRuntimeService()
backend = service.least_busy(min_num_qubits=9, operational=True, simulator=False)
print(backend)
# transpile the circuit for the backend
transpiled_circuit = transpile(qc_shor, backend, optimization_level=2)

job = backend.run(transpiled_circuit)
result = job.result()
real_counts_shor = result.get_counts()
print(real_counts_shor)

<IBMBackend('ibm_brisbane')>


/var/folders/24/ggylxxcj3xq362xbyt3gkg_00000gn/T/ipykernel_41701/2952755171.py:7: DeprecationWarning: backend.run() and related sessions methods are deprecated  as of qiskit-ibm-runtime 0.23 and will be removed no sooner than 6 months after the release date. More details can be found in the primitives migration guide https://docs.quantum.ibm.com/migration-guides/qiskit-runtime.
  job = backend.run(transpiled_circuit)


{'0': 3384, '1': 616}


### Graph histograms

In [9]:

# plot
fig, axes = subplots(1, 2, figsize=(10, 5))
plot_histogram(real_counts_no_shor, ax=axes[0])
axes[0].set_title("No Shor's Code On Quantum Hardware")

plot_histogram(real_counts_shor, ax=axes[1])
axes[1].set_title("With Shor's Code On Quantum Hardware")

tight_layout()
show()

2024-12-10 20:53:43.590 python[41701:3795398] The class 'NSSavePanel' overrides the method identifier.  This method is implemented by class 'NSWindow'


## Analysis

### Auxiliary Functions

In [14]:
def analyze_counts(counts, intended_state="0"):
    """
    Simplifies the process of majority voting for the bitstring since we didn't add the Toffoli gates. Also, calculates fidelity/error.
    Args:
        counts (dict): Measurement results as {bitstring: count}.
        intended_state (str): Logical qubit state intended (default is "0").
    
    Returns:
        dict: Summary including total shots, correct counts, incorrect counts, fidelity, and error percentage.
    """
    correct_count = 0
    total_count = sum(list(counts.values())) # force to a list to avoid that pesky TypeError
    
    for bitstring, count in counts.items(): # tokenize the key-value pairs
        # Count 0s and 1s to determine majority logical state
        num_zeros = bitstring.count("0")
        num_ones = bitstring.count("1")
        logical_state = "0" if num_zeros > num_ones else "1"
        
        # Update correct count if logical state matches intended state
        if logical_state == intended_state:
            correct_count += count

    # Calculate fidelity and error percentage
    fidelity = correct_count / total_count if total_count > 0 else 0 #avoid division by 0
    error_percentage = (1 - fidelity) * 100  # Convert to percentage

    return {
        "total_count": total_count,
        "correct_count": correct_count,
        "incorrect_count": total_count - correct_count,
        "fidelity": fidelity,
        "error_percentage": error_percentage
    }

# stats
print(f'3-Bit Repetition Code Stats: {analyze_counts(counts_corrected)}')
print(f'No 3-Bit Repetition Code Stats: {analyze_counts(counts_uncorrected)}')
print(f'Shor\'s Code Simulated Stats: {analyze_counts(counts_shor)}')
print(f'No Shor\'s Code Simulated Stats: {analyze_counts(counts_no_shor)}')
print(f'Shor\'s Code Stats: {analyze_counts(real_counts_shor)}')
print(f'No Shor\'s Code Stats: {analyze_counts(real_counts_no_shor)}')


3-Bit Repetition Code Stats: {'total_count': 1024, 'correct_count': 987, 'incorrect_count': 37, 'fidelity': 0.9638671875, 'error_percentage': 3.61328125}
No 3-Bit Repetition Code Stats: {'total_count': 1024, 'correct_count': 924, 'incorrect_count': 100, 'fidelity': 0.90234375, 'error_percentage': 9.765625}
Shor's Code Simulated Stats: {'total_count': 1024, 'correct_count': 1024, 'incorrect_count': 0, 'fidelity': 1.0, 'error_percentage': 0.0}
No Shor's Code Simulated Stats: {'total_count': 1024, 'correct_count': 555, 'incorrect_count': 469, 'fidelity': 0.5419921875, 'error_percentage': 45.80078125}
Shor's Code Stats: {'total_count': 4000, 'correct_count': 3384, 'incorrect_count': 616, 'fidelity': 0.846, 'error_percentage': 15.400000000000002}
No Shor's Code Stats: {'total_count': 4000, 'correct_count': 1948, 'incorrect_count': 2052, 'fidelity': 0.487, 'error_percentage': 51.300000000000004}
